In [1]:
from modules import data
from modules import metrics
import pandas
from statistics import mean
import math

In [3]:
df = data.get_data()
df = df.iloc[1:]
df = df.iloc[:4]

In [33]:
# gets average growth from the past few years. 
# function ingests a list of values
# function returns a 1 value

def get_avg_growth(history):
    growth_history = []
    
    #calculate the growth between each year. add to the list
    for i in range(0,(len(history)-1)):
        growth = (history[i] - history[i+1]) / abs(history[i+1])    
        growth_history.append(growth)
    
    #get the average growth
    average_growth_rate = 1 + mean(growth_history)
    return average_growth_rate


# gets cagr growth from the past few years. 
# function ingests a list of values
# function returns a 1 value

def get_cagr_growth(history):
    try:
        base = history[0] / history[-1]
        power = 1 / (len(history) - 1)
        average_growth_rate = math.pow(base, power)
        return average_growth_rate
    except ValueError:
        print("problem occured. Base year is negative value")
        return 0


# gets minimum growth rate from the past few years. If it is less than 1, assume no growth
# function ingests a list of values
# function returns a 1 value
def get_min_growth(history):
    growth_history = []
    
    #calculate the growth between each year. add to the list
    for i in range(0,(len(history)-1)):
        growth = (history[i] - history[i+1]) / abs(history[i+1])    
        growth_history.append(growth)

    #get the minimum growth
    if min(growth_history) >= 0: 
        min_growth_rate = 1 + min(growth_history)
    else: 
        min_growth_rate = 1
    
    return min_growth_rate

In [6]:
# get list of future values via the growth rates  
# function ingests a list of past values, average growth rate, and capped average growth rate
# function returns a list of 10 future value

def get_future_values(history,average_growth_rate,average_growth_rate_cap=1.15):
    future_value = []
    start_value = history[0]
    for i in range(1, 11):
        #for year 1 - 3, grow by average growth rate
        if i <= 3:
            value =  start_value * average_growth_rate
            future_value.append(value)
            start_value = value
        #for year 4-10, grow by cap growth rate
        elif i > 3:
            value =  start_value * average_growth_rate_cap
            future_value.append(value)
            start_value = value
    return future_value

In [7]:
# converts future values into present value and calculate value per share
# function ingests a list of future values values
# function returns get 1 number : value per share

def get_present_values_per_share(future_value, wacc = 1.1, no_of_shares=100000000):
    present_value = 0 

    for i in range(0, len(future_value)):
        value = future_value[i] / pow(wacc, (i+1))
        present_value += value
    
    #calculate present value per share
    fair_value = present_value/no_of_shares
    
    return fair_value

In [8]:
def get_ave_growth_DCF_value(history, cap_growth = 1.15, wacc = 1.1, no_of_shares=100000000):
    
    #find average growth rate 
    average_growth_rate = get_avg_growth(history)
    
    #for year 4-10, if average growth in year 1-3 is more than 15%, cap it at 15%
    if average_growth_rate > cap_growth: 
        average_growth_rate_cap = cap_growth
    else: 
        average_growth_rate_cap = average_growth_rate

    # get next 10 years of future value
    future_value = get_future_values(history,average_growth_rate,average_growth_rate_cap)

    # bring each future value back to present and get per share value
    fair_value = get_present_values_per_share(future_value, wacc, no_of_shares)
    
    return fair_value

In [9]:
def get_cagr_DCF_value(history, cap_growth = 1.15, wacc = 1.1, no_of_shares=100000000):

    average_growth_rate = get_cagr_growth(history)

    #for year 4-10, if average growth in year 1-3 is more than 15%, cap it at 15%
    if average_growth_rate > cap_growth: 
        average_growth_rate_cap = cap_growth
    else: 
        average_growth_rate_cap = average_growth_rate
        
    # get next 10 years of future value
    future_value = get_future_values(history,average_growth_rate,average_growth_rate_cap)

    # bring each future value back to present and get per share value
    fair_value = get_present_values_per_share(future_value,wacc, no_of_shares)
    
    return fair_value

In [10]:
def perpetual_growth_valuation(history, wacc=1.1):
    
    min_growth_rate = get_min_growth(history)

    #bring to current year and then project for next year. assuming we missed this year's payment
    next_payment = history[0] * min_growth_rate * min_growth_rate 

    #apply formula
    valuation = next_payment / (wacc - min_growth_rate)
    return valuation

In [11]:
wacc = metrics.wacc(df,160.39)
shares = metrics.shares(df)

In [12]:
get_cagr_DCF_value(list(df["Operating_Cash_Flow"]),cap_growth=1.15, wacc=wacc, no_of_shares=shares)

98.2759184358719

In [21]:
df.columns 

Index(['year', 'Revenue', 'Interest_Expense_(Operating)', 'EBT',
       'Income_Tax_Provision', 'Net_Income_Common', 'EPS_(Diluted)',
       'Shares_(Diluted,_Weighted)', 'Gross_Margin', 'Net_Profit_Margin',
       'Free_Cash_Flow_Margin', 'Operating_Cash_Flow_Margin',
       'Cash_and_Short_Term_Investments', 'Total_current_assets',
       'Total_non-current_assets', 'Total_Assets', 'Total_current_liabilities',
       'Total_non-current_liabilities', 'Total_liabilities',
       'Shareholders_Equity_(Total)', 'Total_Debt', 'Operating_Cash_Flow',
       'Investing_cash_flow', 'Equity_Repurchase_(Common,_Net)',
       'Dividends_Paid_(Total)', 'Financing_cash_flow',
       'Stock_Based_Compensation', 'P/E_ratio', 'P/FCF_ratio',
       'P/Operating_CF', 'P/B_ratio', 'Current_Ratio', 'Debt/Assets',
       'Debt/Equity', 'Interest_Coverage', 'ROE', 'ROA', 'ROIC',
       'Book_value_per_Share', 'Free_Cash_Flow', 'dividend_per_share'],
      dtype='object', name='Unnamed: 0')

In [22]:
dividend_history = list(df["dividend_per_share"])
dividend_history
FCF_history = list(df["Free_Cash_Flow"])
FCF_history

[5902000000.0, 6740000000.0, 5494000000.0, 5124000000.0]

In [24]:
get_min_growth(FCF_history)

1

In [27]:
perpetual_growth_valuation(FCF_history,wacc)/shares

98.24226457407141

In [35]:
get_cagr_DCF_value([1,2,2,-0.1,-0.5],cap_growth=1.15,wacc = wacc, no_of_shares=shares)

problem occured. Base year is negative value


0.0